In [1]:
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer

Using TensorFlow backend.


In [2]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
# dataframe = pandas.read_csv("kddforpandatrain.csv")#, header=True)
dataframe = pandas.read_csv("kdd_dataset.csv")#, header=True)  # read the whole 10% dataset into dataframe

# samples 3000 random data points from 500k
dataframe = dataframe.sample(n=500)

# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)

dataset = dataframe_encoded.values

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors


8


In [5]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 19
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', matthews_corrcoef])
    return model

In [6]:
matthews_scorer = make_scorer(matthews_corrcoef)

In [7]:

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
y_pred = cross_val_score(estimator, X, dummy_y, cv=kfold)

cm = confusion_matrix(Y, y_pred)
print(cm)

#put calculations here 
print("total: " + str(cm.sum()))
print("accuracy: " + str(numpy.trace(cm) / cm.sum()))

# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: Expected array-like (array or non-string sequence), got <tf.Tensor 'dense_3_target:0' shape=(?, ?) dtype=float32>